### Faire tous les imports nécessaires

In [1]:
import pandas as pd
import string
from nltk.tokenize import sent_tokenize, word_tokenize
import re

### Importer les données

In [2]:
# les components
test_components_data = pd.read_csv('./data/test_components.csv')
train_components_data = pd.read_csv('./data/train_components.csv')
validation_components_data = pd.read_csv('./data/validation_components.csv')

# ajouts des 2 colonnes de contexte aux components
test_components_data['context1'] = ''
test_components_data['context2'] = ''
train_components_data['context1'] = ''
train_components_data['context2'] = ''
validation_components_data['context1'] = ''
validation_components_data['context2'] = ''

# les speechs
test_speeches_data = pd.read_csv('./data/test_speeches.csv')
train_speeches_data = pd.read_csv('./data/train_speeches.csv')
validation_speeches_data = pd.read_csv('./data/validation_speeches.csv')

# données de statistiques
size_components_data = len(test_components_data) + len(train_components_data) + len(validation_components_data)
print("Nombre de composants : ", size_components_data)

Nombre de composants :  32988


Petite fonction pour les resultats intermédiaires

In [3]:
# fonction permettant de retourner string permettant un print en couleur sur un pourcentage en python
# plus le pourcentage est proche de 0 plus il sera rouge, plus il est proche de 100 plus il sera vert
# le pourcentage sera calculé en fonction de la taille du dataset et d'un chiffre donné
def printResultColor(size_dataset, number):
    pourcentage = number / size_dataset * 100
    if pourcentage < 1:
        # print vert
        return("\033[92m {0:0.3f}%\033[0m".format(pourcentage))
    elif pourcentage < 5:
        # print orange
        return("\033[93m {0:0.3f}%\033[0m".format(pourcentage))
    else:
        # print rouge
        return("\033[91m {0:0.3f}%\033[0m".format(pourcentage))

### Implémentation de la premiere fonction

Cette fonction doit trouver le Text de tous components dans les Speeches. Une fois chaque Text trouvé, il faut completer le context 1 et 2.

Cette fonction renvoie une liste de tous les identifiants des components qui n'ont pas été trouvés et une liste de tous les composants qui ont été trouvés plusieur fois (multiple speeches).

In [4]:
def find1Component(dataset_component, dataset_speeches):
    not_found = []
    multiple_speech = []
    for index, row in dataset_component.iterrows():
        textToFind = row.Text
        speeches = dataset_speeches[dataset_speeches['Speech'].str.find(textToFind) != -1]
        # verifier que le component est bien dans un speech
        if len(speeches) == 0:
            not_found.append(index)
        elif len(speeches) == 1:
            # trouver le contexte 1 et 2
            # tokenizer le speech à l'aide de nltk
            sentences = sent_tokenize(speeches.iloc[0].Speech)
            # trouver la position de la phrase contenant le component
            position = -1
            for i in range(len(sentences)):
                if sentences[i].find(textToFind) != -1:
                    position = i
                    break
            if position == -1:
                # peut arriver si le component est en plusieurs phrases et donc la tokenizer ne le trouve pas
                not_found.append(index)
            else:
                # trouver le contexte 1
                if position > 0:
                    dataset_component.at[index, 'context1'] = sentences[position - 1]
                # ajouter la phrase contenant le component au contexte 1
                dataset_component.at[index, 'context1'] += sentences[position]
                # ajouter la phrase qui suit le component au contexte 1
                if position < len(sentences) - 1:
                    dataset_component.at[index, 'context1'] += sentences[position + 1]
                
                # mettre tout le speech dans le contexte 2
                dataset_component.at[index, 'context2'] = speeches.iloc[0].Speech
        else:
            # plusieurs speeches contiennent le component
            # il va falloir verifier l'id du speech et de la section pour verifier si c'est le bon
            multiple_speech.append(index)
    return not_found, multiple_speech

In [5]:
# tester la fonction
not_founded_test = []
multiple_founded_test = []
not_founded_test, multiple_founded_test = find1Component(test_components_data, test_speeches_data)
print("Nombre de composants test non trouvés: {} -> {}".format(len(not_founded_test), printResultColor(len(test_components_data), len(not_founded_test))))
print("Nombre de composants test trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_test), printResultColor(len(test_components_data), len(multiple_founded_test))))

not_founded_train = []
multiple_founded_train = []
not_founded_train, multiple_founded_train = find1Component(train_components_data, train_speeches_data)
print("Nombre de composants train non trouvés: {} -> {}".format(len(not_founded_train), printResultColor(len(train_components_data), len(not_founded_train))))
print("Nombre de composants train trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_train), printResultColor(len(train_components_data), len(multiple_founded_train))))

not_founded_validation = []
multiple_founded_validation = []
not_founded_validation, multiple_founded_validation = find1Component(validation_components_data, validation_speeches_data)
print("Nombre de composants validation non trouvés: {} -> {}".format(len(not_founded_validation), printResultColor(len(validation_components_data), len(not_founded_validation))))
print("Nombre de composants validation trouvés dans plusieurs discours: {} -> {}\n".format(len(multiple_founded_validation), printResultColor(len(validation_components_data), len(multiple_founded_validation))))


Nombre de composants test non trouvés: 102 ->  1.074%
Nombre de composants test trouvés dans plusieurs discours: 1030 ->  10.850%

Nombre de composants train non trouvés: 306 ->  1.793%
Nombre de composants train trouvés dans plusieurs discours: 777 ->  4.552%

Nombre de composants validation non trouvés: 109 ->  1.696%
Nombre de composants validation trouvés dans plusieurs discours: 144 ->  2.241%



### Implémentation de la deuxième fonction

Cette fonction doit trouver le Text de tous les components qui sont en plusieurs phrases et qui n'ont pas été trouvés précédements, avec ou sans la ponctuation. Le resultat renvoyé sera similaire à celui de la fonction précédente.

In [6]:
def find2Components(idsComponents, dataset_component, dataset_speeches):
    not_found = []
    # pour chaque id de component
    for idComponent in idsComponents:
        # get le text du component
        textToFind = re.sub(r'[\.!\?]?( -)?', '', dataset_component.iloc[idComponent].Text)
        # find the text in the speeches, but this time we remove the ponctuations of the speeches using replace function of pandas
        speeches = dataset_speeches[dataset_speeches['Speech'].str.replace(r'[\.!\?]?( -)?', '', regex=True).str.find(textToFind) != -1]
        if(len(speeches) == 1):
            if(idComponent == 187):
                print("ici")
            speeches = speeches.iloc[0]
            # si on a trouvé le component dans un seul speech
            # tokenizer le component en faisant un split sur les espaces
            words_component = dataset_component.iloc[idComponent].Text.split(" ")
            full_component = ""
            for word in words_component:
                if(speeches.Speech.find(full_component) != -1):
                    # si le component est dans le texte
                    # on ajoute le mot suivant au component
                    if(len(full_component) > 0):
                        full_component += " "+word
                    else:
                        full_component = word
                else:
                    # il faut enlever le dernier mot du component
                    # car il manque surement une ponctuation
                    last = full_component[full_component.rfind(" "):]
                    # remove the last word from the component
                    full_component = full_component[:full_component.rfind(" ")]
                    letter = speeches.Speech[speeches.Speech.find(full_component)+len(full_component)]
                    if(speeches.Speech.find(full_component+letter+last) != -1):
                        full_component += letter+last+" "+word
                    elif(speeches.Speech.find(full_component+" "+letter+last) != -1):
                        full_component += " "+letter+last+" "+word
        else:
            not_found.append(idComponent)
    print("Nombre de composants trouvés {} sur {}".format(len(idsComponents) - len(not_found), len(idsComponents)))
    return not_found

index 187 de component train:

<font color="orange"> I don't think it was helpful when he suggested that seventeen million people go to bed hungry every night in the United States</font>

Le component du dataset est incomplet, voici le text original (id: 22 - speech 2 section 11 by Nixon):

I did. And as I pointed out in 1952, (...) I don't think he should say that our prestige is at an all-time low. I think this is very harmful at a time Mr. Khrushchev is here - harmful because it's wrong. <font color="orange">I don't think it was helpful when he suggested</font><font color="red"> - and I'm glad he's corrected this to an extent - </font><font color="orange">that seventeen million people go to bed hungry every night in the United States</font>. Now this just wasn't true. Now, there are people who go to bed hungry in the United States - far less, incidentally, than used to go to bed hungry when we came into power at the end of the Truman Administration. But the thing that is right about the United States, it should be emphasized, (...)


Une partie du texte est manquant dans le speech.

In [7]:
find2Components(not_founded_train, train_components_data, train_speeches_data)

Nombre de composants trouvés 201 sur 306


[187,
 188,
 189,
 191,
 192,
 193,
 194,
 195,
 1196,
 1715,
 2060,
 2141,
 2145,
 2146,
 2210,
 2213,
 2246,
 2302,
 2315,
 2407,
 2469,
 2509,
 2546,
 2578,
 2791,
 2797,
 2940,
 3129,
 3142,
 3149,
 3158,
 3445,
 3477,
 3822,
 3878,
 3986,
 4150,
 4446,
 4703,
 5026,
 5036,
 5040,
 5325,
 5376,
 5377,
 5600,
 5606,
 5632,
 5755,
 5809,
 5880,
 5883,
 5917,
 5984,
 5985,
 6051,
 6091,
 6101,
 8325,
 9264,
 9533,
 9824,
 9905,
 10109,
 10119,
 10835,
 10838,
 10993,
 11085,
 13002,
 13088,
 13292,
 13699,
 13761,
 13886,
 14027,
 14111,
 14181,
 14189,
 14283,
 14285,
 14341,
 14371,
 14459,
 14511,
 14555,
 14634,
 14821,
 14823,
 14836,
 14837,
 14896,
 14920,
 14934,
 14940,
 14944,
 14969,
 14971,
 14972,
 14973,
 14974,
 15578,
 16755,
 16962,
 16965]

In [8]:
# test avec 3 phrases
text = "Je suis une phrase, avec des virgules aussi ?! Je suis une autre phrase ? Je suis la dernière phrase !"
# exemple de component en plusieurs phrases sans ponctuation
component_test = "une phrase, avec des virgules aussi Je suis une autre"

# tokenize en splitant sur les espaces
component_test_words = component_test.split(" ")
full_component = ""
# pour chaque mot du component
for word in component_test_words:
    if(text.find(full_component) != -1):
        # si le component est dans le texte
        # on ajoute le mot suivant au component
        if(len(full_component) > 0):
            full_component += " "+word
        else:
            full_component = word
    else:
        # il faut enlever le dernier mot du component
        # car il manque surement une ponctuation
        last = full_component[full_component.rfind(" "):]
        # remove the last word from the component
        full_component = full_component[:full_component.rfind(" ")]
        letter = text[text.find(full_component)+len(full_component)]
        if(text.find(full_component+letter+last) != -1):
            full_component += letter+last+" "+word
print("Component trouvé: {}".format(full_component))

print("test "+text.replace(r'[\.!\?]', ''))

Component trouvé: une phrase, avec des virgules aussi
test Je suis une phrase, avec des virgules aussi ?! Je suis une autre phrase ? Je suis la dernière phrase !
